In [1]:
from __future__ import print_function
from random import Random

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter


import mnist
from server import Server
from clients import Client
from modules import Net
from allocateGPU import *

allocate_gpu()


class Arguments():
    def __init__(self):
        self.batch_size=64
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.01
        self.momentum=0.5
        self.no_cuda=False
        self.seed=1
        self.log_interval=10
        self.num_clients=10
        self.output_folder='model'
        self.loader_type='non_overlap_label'
        self.loader_path='./data/non_overlap_loader.pk'
args=Arguments()
torch.manual_seed(args.seed)


device='cuda'

# writer=SummaryWriter('./logs/{0}/fed_Med_non_overlap'.format(args.output_folder))



trainData=mnist.train_dataloader(args.num_clients,loader_type=args.loader_type,path=args.loader_path)
testData=mnist.test_dataloader(args.test_batch_size)

#create server instance
model0 = Net().to(device)
server=Server(model0,testData,device)

#create clients instance
for i in range(args.num_clients):
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
    client_i=Client(i,model,trainData[i],optimizer,device)
    server.attach(client_i)

server.test()
for j in range(1):
    print('\n\n########EPOCH %d ########'%j)
    print('###Model distribution###\n')
    server.distribute()
#         group=Random().sample(range(5),1)
    group=range(args.num_clients)
    server.train(group)
    server.test()


Number of gpu avaliable:	1
Current GPU:	0
GPU name: 	GeForce RTX 2080 Ti

Test set: Average loss: 2.3022, Accuracy: 933/10000 (9%)



########EPOCH 0 ########
###Model distribution###

client 0 ## Train Epoch: 0 [0/5923 (0%)]	Loss: 2.273786
client 0 ## Train Epoch: 0 [1610/5923 (98%)]	Loss: 0.000933
client 1 ## Train Epoch: 0 [0/6742 (0%)]	Loss: 2.306377


KeyboardInterrupt: 

In [5]:
from copy import deepcopy
clients=[server.clients[i] for i in group]
Delta=deepcopy(server.emptyStates)
deltas=[c.getDelta() for c in clients]
# def func(arr):
#     return torch.mean(arr)
# for param in Delta:

#     ##stacking the weight in the innerest dimension
#     param_stack=torch.stack([delta[param] for delta in deltas],-1)
#     shaped=param_stack.view(-1,len(clients))
#     ##applying `func` to every array (of size `num_clients`) in the innerest dimension
#     buffer=torch.stack(list(map(func,[shaped[i] for i in range(shaped.size(0))]))).reshape(Delta[param].shape)
#     Delta[param]=buffer
# return Delta

In [11]:
ts=next(iter(trainData[0]))[1]

tensor([1, 8, 7, 1, 1, 5, 8, 9, 8, 6, 1, 7, 8, 7, 5, 8, 4, 2, 9, 9, 6, 8, 6, 9,
        1, 2, 4, 4, 4, 5, 8, 6, 1, 5, 6, 7, 3, 6, 4, 5, 8, 9, 9, 4, 4, 6, 2, 6,
        6, 1, 6, 2, 3, 9, 1, 5, 1, 8, 4, 2, 5, 2, 5, 1, 7, 7, 3, 7, 7, 3, 4, 7,
        6, 2, 5, 3, 5, 1, 2, 9, 3, 3, 6, 1, 9, 6, 4, 7, 5, 6, 1, 7, 4, 1, 8, 3,
        5, 6, 1, 1, 1, 8, 7, 7, 3, 5, 1, 7, 1, 3, 8, 1, 2, 7, 8, 5, 8, 8, 4, 5,
        8, 8, 2, 2, 7, 5, 7, 8])

In [6]:
Delta[next(iter(Delta))]

tensor([[[[0., -0., -0., 0., -0.],
          [0., -0., 0., 0., -0.],
          [0., 0., 0., -0., -0.],
          [-0., 0., -0., 0., 0.],
          [-0., -0., -0., -0., -0.]]],


        [[[0., 0., 0., -0., 0.],
          [0., 0., 0., -0., -0.],
          [-0., 0., -0., 0., -0.],
          [0., -0., 0., 0., -0.],
          [0., -0., 0., -0., -0.]]],


        [[[-0., -0., 0., -0., 0.],
          [0., -0., -0., 0., 0.],
          [-0., 0., 0., 0., -0.],
          [-0., -0., 0., -0., -0.],
          [0., -0., 0., 0., 0.]]],


        [[[0., 0., 0., -0., 0.],
          [-0., -0., 0., -0., 0.],
          [-0., -0., 0., -0., 0.],
          [0., -0., 0., 0., -0.],
          [-0., 0., 0., -0., -0.]]],


        [[[0., -0., 0., 0., -0.],
          [-0., 0., 0., 0., 0.],
          [0., 0., 0., -0., 0.],
          [0., 0., 0., -0., -0.],
          [-0., 0., 0., -0., 0.]]],


        [[[-0., -0., -0., 0., 0.],
          [-0., 0., 0., 0., 0.],
          [0., 0., 0., -0., -0.],
          [-0., -0., 

In [ ]:
deltas

In [2]:
param=next(iter(Delta))

NameError: name 'Delta' is not defined

In [18]:
stacked=torch.stack([delta[param] for delta in deltas],-1)

In [ ]:
deltas[1][param].shape

In [ ]:
torch.median(torch.stack([delta[param] for delta in deltas],-1),-1)[0].shape

In [ ]:
import collections
stat=collections.Counter(torch.median(torch.stack([delta[param] for delta in deltas],-1),-1)[1].flatten().cpu().numpy())

In [ ]:
import matplotlib.pyplot as plt
plt.bar(list(stat.keys()),list(stat.values()))

In [ ]:
torch.median(torch.stack([delta[param] for delta in deltas],-1),-1)

In [14]:
a=Delta[param]

In [28]:
stacked

tensor([[[[[ 2.7868e-04, -2.1018e-05,  1.1906e-03,  ...,  9.4495e-04,
            -1.0599e-03, -1.6563e-04],
           [ 4.6284e-04,  4.5113e-04,  1.2532e-03,  ...,  7.6476e-04,
            -7.3201e-04, -3.2668e-04],
           [ 4.7423e-04,  3.0078e-04,  1.0887e-03,  ...,  6.7870e-04,
            -4.0356e-04, -5.0095e-04],
           [ 3.3353e-04,  9.8787e-05,  7.5781e-04,  ...,  5.7264e-04,
            -2.0116e-04, -4.5139e-04],
           [ 5.6013e-05, -2.1160e-06,  1.2490e-04,  ...,  1.8795e-04,
             8.0794e-05, -1.2194e-04]],

          [[ 1.4518e-04,  1.1408e-04,  1.7709e-03,  ...,  1.4770e-03,
            -1.0331e-03, -3.1900e-04],
           [ 3.8338e-04,  4.5057e-04,  1.6775e-03,  ...,  1.1039e-03,
            -5.1875e-04, -6.3810e-04],
           [ 4.4349e-04,  2.3513e-04,  1.5976e-03,  ...,  1.0616e-03,
            -1.2652e-04, -7.8626e-04],
           [ 2.8501e-04,  7.9200e-06,  1.0339e-03,  ...,  8.0405e-04,
             1.8661e-04, -7.2037e-04],
           [ 8.99

In [38]:
shaped=stacked.view(-1,10)
shaped

tensor([[ 2.7868e-04, -2.1018e-05,  1.1906e-03,  ...,  9.4495e-04,
         -1.0599e-03, -1.6563e-04],
        [ 4.6284e-04,  4.5113e-04,  1.2532e-03,  ...,  7.6476e-04,
         -7.3201e-04, -3.2668e-04],
        [ 4.7423e-04,  3.0078e-04,  1.0887e-03,  ...,  6.7870e-04,
         -4.0356e-04, -5.0095e-04],
        ...,
        [ 1.7797e-03,  1.1671e-03,  7.4675e-04,  ...,  1.0823e-03,
          6.7321e-04, -4.3681e-04],
        [ 1.7763e-03,  5.8950e-04,  5.2349e-04,  ...,  6.7274e-04,
          9.5096e-04, -3.5419e-04],
        [ 2.0004e-03, -1.4410e-04,  3.4934e-04,  ...,  2.6587e-04,
          1.5259e-03, -3.1603e-04]], device='cuda:0')

In [55]:
torch.min(torch.mean(stacked,-1).cpu()-buffer.reshape(a.shape))

tensor(0.)

tensor([[[[ 6.3932e-05,  1.5957e-04,  1.4742e-04,  9.7655e-05,  5.4702e-06],
          [ 1.5839e-04,  2.0434e-04,  1.9241e-04,  1.0846e-04,  2.6545e-05],
          [ 1.6046e-04,  1.8630e-04,  1.9761e-04,  1.2795e-04,  6.8022e-05],
          [ 1.1381e-04,  1.0661e-04,  1.1241e-04,  1.2453e-04,  1.1778e-04],
          [ 3.2256e-05,  1.9905e-05,  3.1298e-05,  7.6298e-05,  1.1685e-04]]],


        [[[ 5.9144e-04,  7.9836e-04,  5.6216e-04,  8.4502e-05, -4.6547e-05],
          [ 6.3055e-04,  7.9968e-04,  5.0821e-04,  7.8207e-05, -1.7126e-05],
          [ 6.4675e-04,  7.3862e-04,  4.3977e-04,  1.5393e-04,  7.5084e-05],
          [ 6.2770e-04,  5.9839e-04,  3.6179e-04,  1.6214e-04,  1.1142e-04],
          [ 4.9421e-04,  4.0914e-04,  2.3910e-04,  1.2446e-04,  1.1968e-04]]],


        [[[-2.0926e-05,  1.3042e-04,  4.1635e-04,  5.9052e-04,  5.7908e-04],
          [ 2.8580e-06,  2.0344e-04,  4.5664e-04,  6.1135e-04,  5.0702e-04],
          [ 1.3143e-06,  1.8668e-04,  3.6382e-04,  4.4904e-04,  3.16

In [77]:
%%time
torch.stack(list(map(func,[shaped[i] for i in range(shaped.size(0))]))).reshape(a.shape)

CPU times: user 17.1 ms, sys: 46 µs, total: 17.1 ms
Wall time: 16.2 ms


tensor([[[[ 6.3932e-05,  1.5957e-04,  1.4742e-04,  9.7655e-05,  5.4702e-06],
          [ 1.5839e-04,  2.0434e-04,  1.9241e-04,  1.0846e-04,  2.6545e-05],
          [ 1.6046e-04,  1.8630e-04,  1.9761e-04,  1.2795e-04,  6.8022e-05],
          [ 1.1381e-04,  1.0661e-04,  1.1241e-04,  1.2453e-04,  1.1778e-04],
          [ 3.2256e-05,  1.9905e-05,  3.1298e-05,  7.6298e-05,  1.1685e-04]]],


        [[[ 5.9144e-04,  7.9836e-04,  5.6216e-04,  8.4502e-05, -4.6547e-05],
          [ 6.3055e-04,  7.9968e-04,  5.0821e-04,  7.8207e-05, -1.7126e-05],
          [ 6.4675e-04,  7.3862e-04,  4.3977e-04,  1.5393e-04,  7.5084e-05],
          [ 6.2770e-04,  5.9839e-04,  3.6179e-04,  1.6214e-04,  1.1142e-04],
          [ 4.9421e-04,  4.0914e-04,  2.3910e-04,  1.2446e-04,  1.1968e-04]]],


        [[[-2.0926e-05,  1.3042e-04,  4.1635e-04,  5.9052e-04,  5.7908e-04],
          [ 2.8580e-06,  2.0344e-04,  4.5664e-04,  6.1135e-04,  5.0702e-04],
          [ 1.3143e-06,  1.8668e-04,  3.6382e-04,  4.4904e-04,  3.16

In [78]:
%%time
buffer=torch.zeros(shaped.size(0))
for i in range(shaped.size(0)):
    buffer[i]=func(shaped[i])
buffer.reshape(a.shape)

CPU times: user 21.3 ms, sys: 8.08 ms, total: 29.4 ms
Wall time: 28.4 ms


tensor([[[[ 6.3932e-05,  1.5957e-04,  1.4742e-04,  9.7655e-05,  5.4702e-06],
          [ 1.5839e-04,  2.0434e-04,  1.9241e-04,  1.0846e-04,  2.6545e-05],
          [ 1.6046e-04,  1.8630e-04,  1.9761e-04,  1.2795e-04,  6.8022e-05],
          [ 1.1381e-04,  1.0661e-04,  1.1241e-04,  1.2453e-04,  1.1778e-04],
          [ 3.2256e-05,  1.9905e-05,  3.1298e-05,  7.6298e-05,  1.1685e-04]]],


        [[[ 5.9144e-04,  7.9836e-04,  5.6216e-04,  8.4502e-05, -4.6547e-05],
          [ 6.3055e-04,  7.9968e-04,  5.0821e-04,  7.8207e-05, -1.7126e-05],
          [ 6.4675e-04,  7.3862e-04,  4.3977e-04,  1.5393e-04,  7.5084e-05],
          [ 6.2770e-04,  5.9839e-04,  3.6179e-04,  1.6214e-04,  1.1142e-04],
          [ 4.9421e-04,  4.0914e-04,  2.3910e-04,  1.2446e-04,  1.1968e-04]]],


        [[[-2.0926e-05,  1.3042e-04,  4.1635e-04,  5.9052e-04,  5.7908e-04],
          [ 2.8580e-06,  2.0344e-04,  4.5664e-04,  6.1135e-04,  5.0702e-04],
          [ 1.3143e-06,  1.8668e-04,  3.6382e-04,  4.4904e-04,  3.16

In [45]:
def func(arr):
    return torch.mean(arr)

In [80]:
s=shaped[0]

In [81]:
torch.median(s)

tensor(-2.1018e-05, device='cuda:0')

In [ ]:
def FedMedian(self,clients):
    Delta=deepcopy(self.emptyStates)
    deltas=[c.getDelta() for c in clients]

    for param in Delta:
        ##stacking the weight in the innerest dimension
        param_stack=torch.stack([delta[param] for delta in deltas],-1)
        Delta[param]=torch.median(param_stack,-1)[0]
    return Delta